In [2]:
from datetime import date
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium import webdriver # driving the actions, links up with the browser
from bs4 import BeautifulSoup
import time
import re
from pick import pick
import progressbar
import os

In [3]:
# change the path

search_webpage = 'https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm'
PATH = "D:\\Program Files (x86)\\chromedriver.exe"
#driver = webdriver.Chrome(PATH)
#driver.get(search_webpage)

In [4]:
yes = ["Yes", "Y", "YES", "y", "yes"]
starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
dashboard_url = "http://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
login_url = 'https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm'
output_name = 'WW_postings.csv'
choice = 'Application Deadlines in the next 10 Days'
potential_html1 = None
potential_html2 = None
potential_html3 = None

In [5]:
{"Work Term Duration:": None, # might need to do some regex filtering
                                         "Job Summary:" : None,
                                         "Job Responsibilities:": None,
                                         "Required Skills:" : None,
                                         "Targeted Degrees and Disciplines:": None,
                                         "Application Documents Required:": None} 

{'Work Term Duration:': None,
 'Job Summary:': None,
 'Job Responsibilities:': None,
 'Required Skills:': None,
 'Targeted Degrees and Disciplines:': None,
 'Application Documents Required:': None}

In [6]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [7]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [ ]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])

def main():
    starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"

    output_name = 'WW_postings.csv'
    
    browser = webdriver.Chrome(PATH)
    browser.get(starting_page) # in original script, this was starting page

    input('Are you ready?')

    
    data = get_job_lists(choice, browser, output_name)

    print("Done!")


def get_job_lists(choice, browser, output_name):
    
    print("Getting job lists ...")

    # Navigate to the job listings page then wait for the javascript to load
    browser.find_element_by_link_text(choice).click()

    input("Are you ready again?")
    
    # Get the HTML of the page and check if this is the last page (next_page_buttons)
    page_html = browser.execute_script("return document.body.innerHTML")
    pattern = r'<a href=".+?" onclick="loadPostingTable(.+?)">\s*»\s*<\/a>'
    next_page_buttons = []
    next_page_buttons = re.findall(pattern, page_html)



    # bar = progressbar.ProgressBar(max_value=self.job_lists_page_count)
    # bar.update(self.gather_current_progress)


    # dots for loading screen
    dots = ""
    counter_page = 0
    # Scrape the tables and save it to output_name as a CSV
    job_posting_importants_order = ['Work Term Duration:', "Job Summary:", "Job Responsibilities:", "Required Skills:", 
                                    "Targeted Degrees and Disciplines:", "Application Documents Required:"]
    with open(output_name, 'w') as f:
        row_count = 0
        job_count = 0
        while (counter_page < 3):
            counter_page += 1
            # loading screen stuff
            os.system('clear')
            print("Working" + dots)
            # update dots
            if (dots == "..."):
                dots = ""
            else:
                dots += "."

            # wait for JavaScript to load then download the page info
            time.sleep(2)
            soup = BeautifulSoup(page_html, "html.parser")
            
            for tr in soup.find_all('tr')[2:]: # iterating throw all rows
            
                
                job_count += 1
                print(job_count)
                try:
                    # all the data that was not in the original script
                    job_posting_importants = {"Work Term Duration:": None, # might need to do some regex filtering
                                             "Job Summary:" : None,
                                             "Job Responsibilities:": None,
                                             "Required Skills:" : None,
                                             "Targeted Degrees and Disciplines:": None,
                                             "Application Documents Required:": None} 


                    tds = tr.find_all('td') # iterating all cells in each row.
                    count = 0
                    job_info_list = []
                    for x in tds[0:-1]: # starting with the 3 buttons


                        if (count == 0): # at 3 buttons
                            link_container = x.find_all('a')[2]

                            onclick_link = link_container['onclick']

                            # clicking the onclick link:
                            browser.execute_script(onclick_link)
                            browser.switch_to.window(browser.window_handles[1])

                            # give the browser 1 seconds for the website to catch up with the code
                            browser.implicitly_wait(2)

                            time.sleep(3)
                            # do stuff, locate the texts of 'job posting importants'

                            #saving the contents of the html of the new page:
                            indiv_html = browser.execute_script("return document.body.innerHTML")
                            indiv_posting = BeautifulSoup(indiv_html, "html.parser")

                            # finding the third 'table' in the page, as the first table is no use to us

                            job_posting_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[1]


                            
                            #print(job_posting_information_table)


                            # iterating through all rows in the table
                            for row in job_posting_information_table.find_all('tr'): 
                                cell_list = row.find_all('td')
                                try:
                                    key = cell_list[0].get_text().strip()
                                except:
                                    continue


                                if (key in job_posting_importants):
                                    try:
                                        job_posting_importants[key] = cell_list[1].get_text().strip()
                                    except:
                                        job_posting_importants[key] = ""


                            application_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[2]
                            job_posting_importants["Application Documents Required:"] = application_information_table.find_all('td')[3].get_text().strip()

                            browser.close() # close current tab
                            browser.switch_to.window(browser.window_handles[0])

                            #print("lala")
                        if (count >= 2):  
                            #print(x.text)
                            job_info_list.append(str(x.text))

                        count += 1


                    for col in job_posting_importants_order:
                        job_info_list.append(job_posting_importants[col])

                    df.loc[row_count] = job_info_list
                    row_count+=1
                except:
                    continue
                
            # navigate to the next page and get the HTML, next page buttons
            next_page_buttons = re.findall(pattern, page_html)
            browser.find_element_by_link_text("»").click()
            time.sleep(2)
            page_html = browser.execute_script("return document.body.innerHTML")


if __name__ == "__main__":
    main()

Are you ready?y
Getting job lists ...
Are you ready again?y
Working
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [ ]:
df

In [ ]:
df.to_csv('viewed.csv')

In [ ]:
ds = df.copy()


In [ ]:
pip install openpyxl


In [ ]:
a = {"bob": 2, 'sam':6, 'asd':123}
a.values()

In [ ]:
' '.join("j, i, s".split())

In [ ]:
e2jin@uwaterloo.ca
Eevee1218.